# Web Scraping para la Recopilación de Datos de Aeropuertos en América

## Introducción:

Este Jupyter Notebook tiene como objetivo principal la recopilación de datos de aeropuertos en América mediante técnicas de web scraping. En el contexto de nuestro proyecto de base de datos orientada a grafos para la gestión de aeropuertos, es esencial contar con información actualizada y precisa sobre los aeropuertos en la región.

El web scraping nos permitirá extraer datos relevantes de fuentes en línea, como sitios web de autoridades de aviación y aerolíneas. Estos datos incluirán detalles sobre ubicaciones geográficas, nombres de aeropuertos, servicios disponibles, rutas operadas y otros atributos esenciales.

## Objetivos:

Obtener datos de aeropuertos en América a partir de fuentes en línea.
Extraer información clave, como nombres de aeropuertos, ubicaciones y detalles operativos.
Almacenar los datos recopilados en un formato adecuado para su posterior análisis y uso en nuestra base de datos orientada a grafos.

## Aviso sobre los Links del Web-Scrapping

**Nota Importante:**
Este cuaderno IPython (Notebook) ha sido creado con el propósito de demostrar las técnicas de web scraping implementadas en el proyecto "Mejora en la Gestión de Aeropuertos en América mediante Base de Datos NoSQL". Durante el desarrollo de la técnica de web scraping con Selenium, se accedió a sitios web que tenían términos y condiciones que prohibían la inclusión de enlaces directos a sus páginas.

Para cumplir con las políticas éticas y legales, se han eliminado los enlaces específicos de las fuentes de información utilizadas en este cuaderno. Sin embargo, es importante destacar que la omisión de los enlaces específicos puede afectar la funcionalidad del código relacionado con Selenium, ya que estaba diseñado específicamente para la estructura de la página web original.

En lugar de proporcionar enlaces directos, se ha conservado el código relacionado con Selenium para fines de demostración y entendimiento de las técnicas utilizadas. Aunque el código puede no funcionar directamente en un entorno diferente al original, su presencia sirve como un ejemplo educativo y muestra la metodología aplicada en el proyecto.

Agradecemos su comprensión y ética en el uso de este material.


## Importación de librerias

* **Requests**: La biblioteca requests le permite al usuario realizar solicitudes HTTP a las páginas web que desee analizar, facilitando la descarga del contenido HTML de dichas páginas para su posterior procesamiento.

* **Beautiful Soup (bs4)**: Beautiful Soup es una herramienta útil para analizar y buscar elementos HTML en el contenido descargado. Permite al usuario buscar y extraer información específica de las páginas web, como títulos, párrafos, enlaces y más.

* **Selenium**: Cuando los sitios web utilizan JavaScript para cargar contenido dinámico, selenium se convierte en una opción valiosa. Con esta biblioteca, el usuario puede automatizar un navegador web para interactuar con el sitio web y extraer datos de páginas que requieren interacción.

* **Pandas**: Pandas es una biblioteca esencial para estructurar y manipular los datos extraídos. Permite al usuario crear DataFrames para organizar los datos en filas y columnas, lo que facilita las operaciones de limpieza, filtrado y procesamiento.

In [1]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import ui
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

## Web Scraping para Recopilación de Datos

En esta sección, se explorará el web scraping, una técnica automatizada para obtener datos de sitios web. Los temas clave incluyen:

* **Solicitudes HTTP**: Se utilizará la biblioteca requests para descargar el contenido HTML de las páginas web.
* **Análisis HTML**: Se empleará Selenium para interactuar con sitios web dinámicos y extraer información relevante.
* **Recopilación de Datos**: Se llevará a cabo la recolección de datos desde los sitios web seleccionados.

In [2]:
chromedriver_autoinstaller.install()  # Instala automáticamente la versión correcta de ChromeDriver

'/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/chromedriver_autoinstaller/118/chromedriver'

In [4]:
def airport_basic(link): #Nos dara la información general del aeropuerto
    
    res = requests.get(link,headers = {'User-Agent': 'Mozilla/5.0'})
    
    soup = bs4.BeautifulSoup(res.text, "html.parser")

    nombre = soup.find_all('strong')[-1].get_text(strip=True) 
    telephone = ' '.join(soup.find_all('li')[-3].text.split()[2:])
    timezone = ' '.join(soup.find_all('li')[-2].text.split()[len(soup.find_all('li')[-2].text.split())-2:])

    data = []
    
    for i in soup.find_all('li')[-4]:
        
        data.append(i)
    
    location = data[2]
    
    info_ae = [nombre, location, timezone, telephone]
    
    return info_ae

In [5]:
def wb_driver(link):
    
    driver = webdriver.Chrome()
    
    driver.get(link)
    
    return driver

In [6]:
def recoleccion(link):
    
    
    info = airport_basic(link)[0].split()[0]
    
    driver = wb_driver(link)
    
    data = []
    
    hora = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="timeStart"]')))

    driver.execute_script("return arguments[0].scrollIntoView(true);", hora)
    
    date_input = driver.find_element(By.ID, "datepicker_input") 

    date_input.click()
    
    time.sleep(2)

    day_9 = driver.find_element(By.XPATH, "//a[text()='9']")
    
    day_9.click()
    
    for i in range(1,25):

        hora_fija = driver.find_element(By.XPATH,f'//*[@id="timeStart"]/option[{i}]')

        hora_fija.click()
        
        time.sleep(10)

        boton_hora = driver.find_element(By.XPATH, '//*[@id="fs_div"]/div[3]/button')

        boton_hora.click()
        
        time.sleep(5)
    
        try:

            vuelos = driver.find_element(By.TAG_NAME, 'tbody')

            vuelos = vuelos.text.split('\n')
            
            a = 0
            
            j = 0
            
            while a != len(vuelos):
            
                arrivo = "NO"

                while arrivo not in ["SCHEDULED","ARRIVED","IN AIR",
                                     'You’re eligible for compensation from this flight. Check what you’re owed for free. Check now',
                                     "UNKNOWN","DETAILS"]:

                    arrivo = vuelos[a]

                    if arrivo in ["SCHEDULED","ARRIVED","IN AIR",
                                  "You’re eligible for compensation from this flight. Check what you’re owed for free. Check now",
                                  "UNKNOWN","DETAILS"]:

                        data.append(vuelos[j:(a+1)])

                        j = a+1

                    a +=1

                    if a == len(vuelos):

                        break


        except NoSuchElementException:

            pass
    
    return data

In [7]:
def get_airlines():
    
    # URL de la página de Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_airline_codes"

    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar la tabla que contiene los códigos IATA
    iata_table = soup.find('table', {'class': 'wikitable'})

    # Inicializar una lista para almacenar los datos de la tabla
    iata_data = []

    # Recorrer las filas de la tabla
    for row in iata_table.find_all('tr')[1:]:  # Ignorar la primera fila que contiene encabezados
        
        # Obtener todas las celdas de la fila
        cells = row.find_all('td')
        
        if len(cells) >= 4:
            
            # Obtener el código IATA y comprobar si no está vacío
            iata_code = cells[0].get_text(strip=True)
            iaco = cells[1].get_text(strip=True)
            
            if iata_code and iaco:
                
                # Si el código IATA ni IACO no está vacío
                airline_name = cells[2].get_text(strip=True)
                call_sign = cells[3].get_text(strip=True)
                country = cells[4].get_text(strip=True)

                # Agregar los datos a la lista
                iata_data.append([iata_code, iaco, airline_name, call_sign,country])
                
    return iata_data

## Creación de DataFrames

Los DataFrames son una estructura tabular proporcionada por la biblioteca de **pandas**, desempeñan un rol central al proporcionar una base sólida desde la cual se insertarán los datos en sistemas de bases de datos, tales como ArangoDB y OrientDB.

### Utilizando pandas para la Creación de DataFrames

La librería **pandas** ofrece un conjunto completo de herramientas y funciones diseñadas para la carga, manipulación y análisis de datos en formato tabular. Durante esta fase del proyecto, se empleará **pandas** para la creación de DataFrames que representarán una variedad de datos, incluyendo información sobre aeropuertos, rutas, aerolíneas y otros atributos relevantes.

La creación de DataFrames con **pandas** asegura una organización y estructura adecuada de los datos antes de que se inserten en bases de datos de grafos, como ArangoDB y OrientDB, que se utilizarán en etapas posteriores del proyecto.

### Preparación para la Inserción en Bases de Datos de Grafos

La creación de DataFrames ayuda a la gestión de datos de aeropuertos. Utilizando **pandas** para esta tarea, se facilita la transición de datos desde una estructura tabular hacia una representación basada en grafos, en ArangoDB como en OrientDB.

Se crearan dos bases de datos en formato csv
1. **Base de Datos de Aeropuertos**: Esta base contiene información general sobre los aeropuertos, como el nombre, ubicación geográfica, zona horaria y número de contacto.

3. **Base de Datos de las Aerolineas**: Esta base contiene información de las aerolíneas; el código IATA, IACO, nombre de la aerolínea, llamada de identifiación, país origen.

2. **Base de Datos de Vuelos**: Esta base se enfoca en detalles específicos de los vuelos, como las horas de salida y llegada, la aerolínea y otros atributos relacionados con los vuelos.

La organización de los DataFrames son importantes para garantizar la calidad de los datos antes de que sean insertados en las bases de datos de grafos de manera correcta.


In [10]:
#Base de Datos de Aeropuertos

df_info_aero = pd.DataFrame(columns=["Nombre","Ubicación","Timezone","Teléfono"],
                           index= [x for x in range(0,len(aeropuertos_links)-1,2)])

for aeropuerto in range(0,len(aeropuertos_links)-1,2):
    
    info = airport_basic(aeropuertos_links[aeropuerto])
    
    df_info_aero.loc[aeropuerto] = [info[0],info[1],info[2],info[3]]
    
    time.sleep(5)
    
df_info_aero.head()

,Nombre,Ubicación,Timezone,Teléfono
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352
2,San Salvador Cuscatlan Airport (SAL),"RN-5, San Salvador, El Salvador",(UTC/GMT -6),+503 2349 9455
4,Panama City Albrook Marcos A. Gelabert Airport...,"Avenida Canfield, Panamá, Panama",(UTC/GMT -5),+507 501-9272
6,Guatemala City La Aurora Airport (GUA),"Guatemala City, Guatemala",(UTC/GMT -6),+502 2321 5000
8,Puerto Rico Airport (PUR),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329


In [11]:
##Exportación de Datos de Aeropuertos a formato csv

df_info_aero.to_csv("Info_Basica_Aeropuertos.csv")

In [73]:
#Base de Datos de las Aerolineas

aerolinea = get_airlines()

df_aerolineas = pd.DataFrame(aerolinea, columns=["IATA", "IACO", "Aerolinea","Llamada de Identificación", "País"])

df_aerolineas.tail(7)

,IATA,IACO,Aerolinea,Llamada de Identificación,País
1433,4Y,UYA,Yute Air Alaska,,United States
1434,2N,UMK,Yuzhmashavia,YUZMASH,Ukraine
1435,C4,IMX,Zimex Aviation,ZIMEX,Switzerland
1436,Q3,MBN,Zambian Airways,ZAMBIANA,Zambia
1437,3J,WZP,Zip,ZIPPER,Canada
1438,ZG,TZP,Zipair Tokyo,ZIPPY,Japan
1439,Z4,OOM,Zoom Airlines,ZOOM,Canada


In [74]:
## Exportación de Base de Datos de las Aerolineas

df_aerolineas.to_csv("Aerolineas.csv")

In [12]:
#Base de Datos de Vuelos

arrivos = pd.DataFrame(columns=["Origen","Destino","Numero de Vuelo", "Salida", "Llegada"])

In [13]:
def iteracion_aeropuerto(aeropuertos_link): #Lista de los url del aeropuerto
    
    a = 0
    
    for k in range(18,len(aeropuertos_links)-1,2):
    
        data = recoleccion(aeropuertos_links[k])
        
        time.sleep(5)
    
        for i in range(len(data)): 
    
    
            if data[i][0]=="Delay":
    
                if data[i][2].split(' ')[0] != "Codeshare":
                    
                    if len(data[i])<=5:
                        
                        pass
    
                    elif data[i][5] != '' and data[i][5][0].isdigit():
                        arrivos.loc[a] = [data[i][3],df_info_aero["Nombre"][k],
                                                 data[i][2],data[i][5], data[i][-3]]
                    else:
                        arrivos.loc[a] = [data[i][3],df_info_aero["Nombre"][k],
                                                 data[i][2],data[i][4].split(' ')[-1], data[i][-3]]
                else:
                    
                    if len(data[i])<6:
                        
                        pass
    
                    elif data[i][6] != '' and data[i][6][0].isdigit():
                        arrivos.loc[a] = [data[i][4],df_info_aero["Nombre"][k],
                                                 data[i][3],data[i][6], data[i][-3]]
                    else:
                        arrivos.loc[a] = [data[i][4],df_info_aero["Nombre"][k],
                                                 data[i][3],data[i][5].split(' ')[-1], data[i][-3]]
            if data[i][0]=='':
    
                pass
    
            else:
    
                if data[i][0].split(' ')[0] != "Codeshare":
                    
                    if len(data[i])<=3:
                        
                        pass
    
                    elif data[i][3] != '' and data[i][3][0].isdigit():
                        arrivos.loc[a] = [data[i][1],df_info_aero["Nombre"][k],
                                                 data[i][0],data[i][3], data[i][-3]]
                    else:
                        arrivos.loc[a] = [data[i][1],df_info_aero["Nombre"][k],
                                                 data[i][0],data[i][2].split(' ')[-1], data[i][-3]]
                else:
                    
                    if len(data[i])<=4:
                        
                        pass
    
                    elif data[i][4] != '' and data[i][4][0].isdigit():
                        arrivos.loc[a] = [data[i][2],df_info_aero["Nombre"][k],
                                                 data[i][1],data[i][4], data[i][-3]]
                    else:
                        arrivos.loc[a] = [data[i][2],df_info_aero["Nombre"][k],
                                                 data[i][1],data[i][3].split(' ')[-1], data[i][-3]]
                        
                a +=1
    return data

ConnectionError: HTTPSConnectionPool(host='airportinfo.live', port=443): Max retries exceeded with url: /arrivals/phl/airport-new-philadelphia-philadelphia (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x149c55810>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [270]:
arrivos.head()

,Origen,Destino,Numero de Vuelo,Salida,Llegada
0,Miami [MIA],Brasilia Distrito Federal Brasilia Airport (BSB),G3 7749,22:00,06:23
3,Lima [LIM],Brasilia Distrito Federal Brasilia Airport (BSB),DL 6326,0:05,06:40
4,Buenos Aires [EZE],Brasilia Distrito Federal Brasilia Airport (BSB),AR 1052,0:55,04:02
5,Rio de Janeiro [GIG],Brasilia Distrito Federal Brasilia Airport (BSB),EK 3677,4:20,06:05
6,Rio de Janeiro [GIG],Brasilia Distrito Federal Brasilia Airport (BSB),KL 9400,4:20,06:05


In [15]:
#Exportamos la Base de Vuelos a formato CSV

arrivos.to_csv("arrivos.csv")

In [14]:
arrivos

,Origen,Destino,Numero de Vuelo,Salida,Llegada
0,Miami [MIA],Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38
1,Bogota [BOG],Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55
2,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16
3,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16
4,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16
...,...,...,...,...,...
16735,Orlando [MCO],Miami Airport (MIA),IB 4728,9:45,
16736,San José [SJO],Miami Airport (MIA),AA 1204,6:00,10:28
16737,San José [SJO],Miami Airport (MIA),IB 4551,6:00,10:28
16738,Detroit [DTW],Miami Airport (MIA),DL 1468,8:00,10:40
